Having spent years analyzing financial records for big banks, you've finally scratched your idealistic itch and joined the education sector. Your latest role is Chief Data Scientist for your city's school district. In this capacity, you'll be helping the school board and mayor make strategic decisions regarding future school budgets and priorities.

As a first task, you've been asked to analyze the district-wide standardized test results. You'll be given access to every student's math and reading scores, as well as various information on the schools they attend. Your task is to aggregate the data to showcase obvious trends in school performance.

In [160]:
import pandas as pd

In [161]:
school_raw = pd.read_csv(r'C:\Users\ericj\OneDrive\Documents\Repositories\School_Budgeting_and_Priorities\Data\schools_complete.csv')
student_raw = pd.read_csv(r'C:\Users\ericj\OneDrive\Documents\Repositories\School_Budgeting_and_Priorities\Data\students_complete.csv')

### District Summary

Create a high-level snapshot, in a DataFrame, of the district's key metrics, including the following:

* Total schools
* Total students
* Total budget
* Average math score
* Average reading score
* % passing math (the percentage of students who passed math)
* % passing reading (the percentage of students who passed reading)
* % overall passing (the percentage of students who passed math AND reading)

In [162]:
school_raw.head()

,School ID,school_name,type,size,budget
0,0,Huang High School,District,2917,1910635
1,1,Figueroa High School,District,2949,1884411
2,2,Shelton High School,Charter,1761,1056600
3,3,Hernandez High School,District,4635,3022020
4,4,Griffin High School,Charter,1468,917500


In [163]:
Number_of_schools = len(school_raw['school_name'].value_counts())

In [164]:
student_raw.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score
0,0,Paul Bradley,M,9th,Huang High School,66,79
1,1,Victor Smith,M,12th,Huang High School,94,61
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58
4,4,Bonnie Ray,F,9th,Huang High School,97,84


In [165]:
Number_of_students = len(student_raw['Student ID'].value_counts())

In [166]:
Total_budget = school_raw['budget'].sum()

In [167]:
Average_math = student_raw['math_score'].mean()

In [168]:
Average_reading = student_raw['reading_score'].mean()

In [169]:
district_summary = pd.DataFrame([{'Total Number of Schools' : Number_of_schools, 
                                  'Total Number of Students' : Number_of_students, 
                                  'Total Budget' : Total_budget, 'Average Math Score in %' : Average_math, 
                                  'Average Reading Score in %' : Average_reading}])

In [170]:
district_summary

,Total Number of Schools,Total Number of Students,Total Budget,Average Math Score in %,Average Reading Score in %
0,15,39170,24649428,78.985371,81.87784


In [171]:
Number_passing_math = student_raw.loc[student_raw['math_score']>=70]['math_score'].count()
Number_passing_reading = student_raw.loc[student_raw['reading_score']>=70]['reading_score'].count()
Number_passing_both = Number_passing_math + Number_passing_reading
Total_math = student_raw.loc[student_raw['math_score']]['math_score'].count()
Total_reading = student_raw.loc[student_raw['reading_score']]['math_score'].count()
Total_math_reading = Total_math + Total_reading


district_summary['Percent Passing Math in %'] = round((Number_passing_math / Total_math * 100), 2)
district_summary['Percent Passing Reading in %'] = round((Number_passing_reading / Total_reading * 100), 2)
district_summary['Percent Passing Both in %'] = round((Number_passing_both / Total_math_reading * 100), 2)

In [172]:
agg_data

,Total Number of Schools,Total Number of Students,Total Budget,Average Math Score,Average Reading Score,Percent Passing Math in %,Percent Passing Reading in %,Percent Passing Both in %
0,15,39170,24649428,78.985371,81.87784,74.98,85.81,80.39


### School Summary

Create a DataFrame that summarizes key metrics about each school, including the following:

* School name
* School type
* Total students
* Total school budget
* Per student budget
* Average math score
* Average reading score
* % passing math (the percentage of students who passed math 70 or greater)
* % passing reading (the percentage of students who passed reading 70 or greater)
* % overall passing (the percentage of students who passed math AND reading)

In [173]:
merged_df = pd.merge(school_raw, student_raw, how = 'left', on = 'school_name')
merged_df

,School ID,school_name,type,size,budget,Student ID,student_name,gender,grade,reading_score,math_score
0,0,Huang High School,District,2917,1910635,0,Paul Bradley,M,9th,66,79
1,0,Huang High School,District,2917,1910635,1,Victor Smith,M,12th,94,61
2,0,Huang High School,District,2917,1910635,2,Kevin Rodriguez,M,12th,90,60
3,0,Huang High School,District,2917,1910635,3,Dr. Richard Scott,M,12th,67,58
4,0,Huang High School,District,2917,1910635,4,Bonnie Ray,F,9th,97,84
...,...,...,...,...,...,...,...,...,...,...,...
39165,14,Thomas High School,Charter,1635,1043130,39165,Donna Howard,F,12th,99,90
39166,14,Thomas High School,Charter,1635,1043130,39166,Dawn Bell,F,10th,95,70
39167,14,Thomas High School,Charter,1635,1043130,39167,Rebecca Tanner,F,9th,73,84
39168,14,Thomas High School,Charter,1635,1043130,39168,Desiree Kidd,F,10th,99,90


In [174]:
group_by_school = merged_df.groupby(['school_name'])
schname = group_by_school['school_name'].first()
schtype = group_by_school['type'].first()
schstu = group_by_school['Student ID'].count()
schbud = group_by_school['budget'].first()
per_stu_bud = schbud / schstu
schmath = round(group_by_school['math_score'].mean(), 2)
schread = round(group_by_school['reading_score'].mean(), 2)
passmath = round((merged_df[merged_df['math_score'] >= 70].groupby('school_name')['Student ID'].count() / schstu) * 100, 2)
passread = round((merged_df[merged_df['reading_score'] >= 70]
                  .groupby('school_name')['Student ID'].count() / schstu) * 100, 2)
passboth = round((merged_df[(merged_df['math_score'] >= 70) & (merged_df['reading_score'] >= 70)]
                  .groupby('school_name')['Student ID'].count() / schstu) * 100, 2)

school_summary = pd.DataFrame({'School Name' : schname, 'School Type' : schtype, 'Total Students' : schstu, 
                               'Total Budget in $' : schbud, 'Budget per Student in $' : per_stu_bud, 
                               'Average Math Score in %' : schmath, 
                               'Average Reading Score in %' : schread, '% Passing Math' : passmath, 
                               '% Passing Reading' : passread, '% Passing Both' : passboth})
school_summary.head()

,School Name,School Type,Total Students,Total Budget in $,Budget per Student in $,Average Math Score in %,Average Reading Score in %,% Passing Math,% Passing Reading,% Passing Both
school_name,,,,,,,,,,
Bailey High School,Bailey High School,District,4976,3124928,628.0,77.05,81.03,66.68,81.93,54.64
Cabrera High School,Cabrera High School,Charter,1858,1081356,582.0,83.06,83.98,94.13,97.04,91.33
Figueroa High School,Figueroa High School,District,2949,1884411,639.0,76.71,81.16,65.99,80.74,53.20
Ford High School,Ford High School,District,2739,1763916,644.0,77.10,80.75,68.31,79.30,54.29
Griffin High School,Griffin High School,Charter,1468,917500,625.0,83.35,83.82,93.39,97.14,90.60


### Highest-Performing Schools (by % Overall Passing)

Create a DataFrame that highlights the top 5 performing schools based on % Overall Passing. Include the following metrics:

* School name
* School type
* Total students
* Total school budget
* Per student budget
* Average math score
* Average reading score
* % passing math (the percentage of students who passed math)
* % passing reading (the percentage of students who passed reading)
* % overall passing (the percentage of students who passed math AND reading)

In [175]:
highest_performing = school_summary.sort_values('% Passing Both', ascending = False).head()
highest_performing

,School Name,School Type,Total Students,Total Budget in $,Budget per Student in $,Average Math Score in %,Average Reading Score in %,% Passing Math,% Passing Reading,% Passing Both
school_name,,,,,,,,,,
Cabrera High School,Cabrera High School,Charter,1858,1081356,582.0,83.06,83.98,94.13,97.04,91.33
Thomas High School,Thomas High School,Charter,1635,1043130,638.0,83.42,83.85,93.27,97.31,90.95
Griffin High School,Griffin High School,Charter,1468,917500,625.0,83.35,83.82,93.39,97.14,90.60
Wilson High School,Wilson High School,Charter,2283,1319574,578.0,83.27,83.99,93.87,96.54,90.58
Pena High School,Pena High School,Charter,962,585858,609.0,83.84,84.04,94.59,95.95,90.54


### Lowest-Performing Schools (by % Overall Passing)

Create a DataFrame that highlights the bottom 5 performing schools based on % Overall Passing. Include the following metrics:

* School name
* School type
* Total students
* Total school budget
* Per student budget
* Average math score
* Average reading score
* % passing math (the percentage of students who passed math)
* % passing reading (the percentage of students who passed reading)
* % overall passing (the percentage of students who passed math AND reading)


In [176]:
lowest_performing = school_summary.sort_values('% Passing Both', ascending = True).head()
lowest_performing

,School Name,School Type,Total Students,Total Budget in $,Budget per Student in $,Average Math Score in %,Average Reading Score in %,% Passing Math,% Passing Reading,% Passing Both
school_name,,,,,,,,,,
Rodriguez High School,Rodriguez High School,District,3999,2547363,637.0,76.84,80.74,66.37,80.22,52.99
Figueroa High School,Figueroa High School,District,2949,1884411,639.0,76.71,81.16,65.99,80.74,53.20
Huang High School,Huang High School,District,2917,1910635,655.0,76.63,81.18,65.68,81.32,53.51
Hernandez High School,Hernandez High School,District,4635,3022020,652.0,77.29,80.93,66.75,80.86,53.53
Johnson High School,Johnson High School,District,4761,3094650,650.0,77.07,80.97,66.06,81.22,53.54


### Math Scores by Grade

Create a DataFrame that lists the average math score for students of each grade level (9th, 10th, 11th, 12th) at each school.

In [180]:
schoolname = merged_df.groupby('school_name')['school_name'].first()
mathnine = round(merged_df[merged_df['grade'] == '9th'].groupby('school_name')['math_score'].mean(), 2)
mathten = round(merged_df[merged_df['grade'] == '10th'].groupby('school_name')['math_score'].mean(), 2)
matheleven = round(merged_df[merged_df['grade'] == '11th'].groupby('school_name')['math_score'].mean(), 2)
mathtwelve = round(merged_df[merged_df['grade'] == '12th'].groupby('school_name')['math_score'].mean(), 2)

math_by_grade = pd.DataFrame({'School Name' : schoolname, 'Grade 9 Average Math Score in %' : mathnine, 
                              'Grade 10 Average Math Score in %' : mathten, 
                             'Grade 11 Average Math Score in %' : matheleven, 'Grade 12 Average Math Score in %': mathtwelve})

math_by_grade = math_by_grade.set_index('School Name')
math_by_grade

,Grade 9 Average Math Score in %,Grade 10 Average Math Score in %,Grade 11 Average Math Score in %,Grade 12 Average Math Score in %
School Name,,,,
Bailey High School,77.08,77.00,77.52,76.49
Cabrera High School,83.09,83.15,82.77,83.28
Figueroa High School,76.40,76.54,76.88,77.15
Ford High School,77.36,77.67,76.92,76.18
Griffin High School,82.04,84.23,83.84,83.36
Hernandez High School,77.44,77.34,77.14,77.19
Holden High School,83.79,83.43,85.00,82.86
Huang High School,77.03,75.91,76.45,77.23
Johnson High School,77.19,76.69,77.49,76.86


### Reading Scores by Grade

Create a DataFrame that lists the average reading score for students of each grade level (9th, 10th, 11th, 12th) at each school.

In [179]:
readnine = round(merged_df[merged_df['grade'] == '9th'].groupby('school_name')['reading_score'].mean(), 2)
readten = round(merged_df[merged_df['grade'] == '10th'].groupby('school_name')['reading_score'].mean(), 2)
readeleven = round(merged_df[merged_df['grade'] == '11th'].groupby('school_name')['reading_score'].mean(), 2)
readtwelve = round(merged_df[merged_df['grade'] == '12th'].groupby('school_name')['reading_score'].mean(), 2)

reading_by_grade = pd.DataFrame({'School Name' : schoolname, 'Grade 9 Average Reading Score in %' : readnine, 
                              'Grade 10 Average Reading Score in %' : readten, 
                             'Grade 11 Average Reading Score in %' : readeleven, 
                              'Grade 12 Average Reading Score in %': readtwelve})

reading_by_grade = reading_by_grade.set_index('School Name')
reading_by_grade

,Grade 9 Average Reading Score in %,Grade 10 Average Reading Score in %,Grade 11 Average Reading Score in %,Grade 12 Average Reading Score in %
School Name,,,,
Bailey High School,81.30,80.91,80.95,80.91
Cabrera High School,83.68,84.25,83.79,84.29
Figueroa High School,81.20,81.41,80.64,81.38
Ford High School,80.63,81.26,80.40,80.66
Griffin High School,83.37,83.71,84.29,84.01
Hernandez High School,80.87,80.66,81.40,80.86
Holden High School,83.68,83.32,83.82,84.70
Huang High School,81.29,81.51,81.42,80.31
Johnson High School,81.26,80.77,80.62,81.23


### Scores by School Spending

Create a table that breaks down school performance based on average spending ranges (per student). Use your judgment to create four bins with reasonable cutoff values to group school spending. Include the following metrics in the table:

* Average math score
* Average reading score
* % passing math (the percentage of students who passed math)
* % passing reading (the percentage of students who passed reading)
* % overall passing (the percentage of students who passed math AND reading)

In [199]:
bins = [560, 585, 610, 635, 660]
group_labels = ['560 to 584.99', '585 to 609.99', '610 to 634.99', '635 to 659.99']
merged_df['Budget Category'] = pd.cut(((merged_df['budget']) / (merged_df['size'])), bins, 
                                           labels = group_labels, include_lowest = True)

group_by_budget = merged_df.groupby('Budget Category')

budget_avg_math = round(group_by_budget['math_score'].mean(), 2)
budget_avg_read = round(group_by_budget['reading_score'].mean(), 2)
budget_total_students = group_by_budget['Student ID'].count()
budget_over_math = merged_df[merged_df['math_score'] >= 70].groupby('Budget Category')['Student ID'].count()
budget_over_reading = merged_df[merged_df['reading_score'] >= 70].groupby('Budget Category')['Student ID'].count()
budget_over_both = merged_df[(merged_df['math_score'] >= 70) & (merged_df['reading_score'] >= 70)].groupby('Budget Category')['Student ID'].count()
budget_pass_math = round(budget_over_math / budget_total_students * 100, 2)
budget_pass_reading = round(budget_over_reading / budget_total_students * 100, 2)
budget_pass_both = round(budget_over_both / budget_total_students * 100, 2)

summary_by_budget = pd.DataFrame({'Average Math Score in %':budget_avg_math, 
                                  'Average Reading Score in %' : budget_avg_read, 
                                  '% Passing Math' : budget_pass_math, '% Passing Reading' : budget_pass_reading, 
                                  '% Passing Both': budget_pass_both})

summary_by_budget



,Average Math Score in %,Average Reading Score in %,% Passing Math,% Passing Reading,% Passing Both
Budget Category,,,,,
560 to 584.99,83.36,83.96,93.70,96.69,90.64
585 to 609.99,83.53,83.84,94.12,95.89,90.12
610 to 634.99,78.48,81.67,72.77,85.40,62.83
635 to 659.99,77.42,81.15,68.34,81.82,56.07


### Scores by School Size

Create a table that breaks down school performance based on school size (small, medium, or large).

In [200]:
bins = [0, 2000, 4000, 6000]
group_names = ['Small or < 2000', 'Medium or 2000 ~ 4000', 'Large or > 4000']
merged_df['Size Category'] = pd.cut(merged_df['size'], bins, labels = group_names, include_lowest = True)

group_by_size = merged_df.groupby('Size Category')

size_avg_math = round(group_by_size['math_score'].mean(), 2)
size_avg_read = round(group_by_size['reading_score'].mean(), 2)
size_total_students = group_by_size['Student ID'].count()
size_over_math = merged_df[merged_df['math_score'] >= 70].groupby('Size Category')['Student ID'].count()
size_over_reading = merged_df[merged_df['reading_score'] >= 70].groupby('Size Category')['Student ID'].count()
size_over_both = merged_df[(merged_df['math_score'] >= 70) & (merged_df['reading_score'] >= 70)].groupby('Size Category')['Student ID'].count()
size_pass_math = round(size_over_math / size_total_students * 100, 2)
size_pass_reading = round(size_over_reading / size_total_students * 100, 2)
size_pass_both = round(size_over_both / size_total_students * 100, 2)

summary_by_size = pd.DataFrame({'Average Math Score in %':size_avg_math, 
                                  'Average Reading Score in %' : size_avg_read, 
                                  '% Passing Math' : size_pass_math, '% Passing Reading' : size_pass_reading, 
                                  '% Passing Both': size_pass_both})

summary_by_size

,Average Math Score in %,Average Reading Score in %,% Passing Math,% Passing Reading,% Passing Both
Size Category,,,,,
Small or < 2000,83.44,83.88,93.66,96.67,90.56
Medium or 2000 ~ 4000,77.81,81.41,70.73,82.87,59.14
Large or > 4000,77.13,80.98,66.50,81.35,53.92


### Scores by School Type

Create a table that breaks down school performance based on type of school (district or charter).

In [201]:
group_by_type = merged_df.groupby('type')

type_avg_math = round(group_by_type['math_score'].mean(), 2)
type_avg_read = round(group_by_type['reading_score'].mean(), 2)
type_total_students = group_by_type['Student ID'].count()
type_over_math = merged_df[merged_df['math_score'] >= 70].groupby('type')['Student ID'].count()
type_over_reading = merged_df[merged_df['reading_score'] >= 70].groupby('type')['Student ID'].count()
type_over_both = merged_df[(merged_df['math_score'] >= 70) & (merged_df['reading_score'] >= 70)].groupby('type')['Student ID'].count()
type_pass_math = round(type_over_math / type_total_students * 100, 2)
type_pass_reading = round(type_over_reading / type_total_students * 100, 2)
type_pass_both = round(type_over_both / type_total_students * 100, 2)

summary_by_type = pd.DataFrame({'Average Math Score in %':type_avg_math, 
                                  'Average Reading Score in %' : type_avg_read, 
                                  '% Passing Math' : type_pass_math, '% Passing Reading' : type_pass_reading, 
                                  '% Passing Both': type_pass_both})

summary_by_type

,Average Math Score in %,Average Reading Score in %,% Passing Math,% Passing Reading,% Passing Both
type,,,,,
Charter,83.41,83.90,93.70,96.65,90.56
District,76.99,80.96,66.52,80.91,53.70
